In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
from torchvision.transforms.functional import pad as PAD
import torch.optim as optim
from torch.utils.data.dataset import random_split
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
import re
import pandas as pd
from sklearn.metrics import classification_report
import wandb
import numbers
import natsort
from PIL import Image
from torchvision.transforms import ToPILImage,ToTensor
ToPILImage = ToPILImage()
os.environ['CUDA_VISIBLE_DEVICES']='1'

from efficientnet_pytorch import EfficientNet

PyTorch Version:  1.4.0


In [2]:
hyperparameter_defaults = dict(
    input_size = 1024,
    batch_size = 1,
)
os.environ['WANDB_NOTEBOOK_NAME']='GCAM Upload Notebook'
wandb.init(project="GCAM Results",config=hyperparameter_defaults)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: szho6430 (use `wandb login --relogin` to force relogin)


In [3]:
from torch.utils.data import Dataset

class TestSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_name = self.total_imgs[idx]
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image,img_name
    
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, fill=0, padding_mode='constant'):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']

        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        return PAD(img, get_padding(img), self.fill, self.padding_mode)
    
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.\
            format(self.fill, self.padding_mode)

In [4]:
test_transform = transforms.Compose([
    NewPad(),
    transforms.Resize(1024),
    transforms.ToTensor()
    ])

In [5]:
DataSet = TestSet(main_dir='../../data/AnimalDatasetV2Original/',transform = test_transform)

In [6]:
dataloader = torch.utils.data.DataLoader(DataSet,batch_size=1,shuffle=False)

In [7]:
LIST=pd.read_csv('./post_meta.csv',dtype={'Image No.':str})
LIST.head()

,Patient No.,Species,Breed,Age (yrs),Gender,Neuter,Image Findings,Thoracic or Pelvic,Image No.,Normal_Abnormal,Test_CV,Img Name,Direct,MURA
0,1,Canine,German Shepherd,2,Female,Yes,Osteochondrosis with Osteophytes,Thoracic,001,False,4,001.jpg,1,1
1,1,Canine,German Shepherd,2,Female,Yes,Osteochondrosis with Osteophytes,Thoracic,002,False,1,002.jpg,1,1
2,1,Canine,German Shepherd,2,Female,Yes,Osteochondrosis with Osteophytes,Thoracic,003,False,3,003.jpg,0,1
3,1,Canine,German Shepherd,2,Female,Yes,Osteochondrosis with Osteophytes,Thoracic,004,False,3,004.jpg,1,1
4,1,Canine,German Shepherd,2,Female,Yes,Osteochondrosis with Osteophytes,Thoracic,005,False,4,005.jpg,1,1


In [8]:
for inputs,name in dataloader:
    inputs = ToPILImage(inputs.squeeze())
    image_no = name[0].replace('.jpeg','').replace('.jpg','')
    Image_Findings = LIST.loc[LIST['Image No.']==image_no,'Image Findings'].item()
    caption = 'Findings in this X-ray:\n'+Image_Findings
    wandb.log({image_no:[wandb.Image(inputs,caption=caption)]})
    print(f"{image_no}\t{Image_Findings}")

/home/dasleo/miniconda3/envs/pytorch13/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.


001	Osteochondrosis with Osteophytes
002	Osteochondrosis with Osteophytes
003	Osteochondrosis with Osteophytes
004	Osteochondrosis with Osteophytes
005	Osteochondrosis with Osteophytes
006	Fracture
007	Fracture
008	Hypertrophic Osteopathy
009	Hypertrophic Osteopathy
010	Fracture
011	Fracture
012	Fracture
013	Fracture
014	Fracture
015	Fracture
016	Normal
017	Normal
018	Normal
019	Normal
020	Osteochondrosis
021	Osteochondrosis
022	Osteochondrosis
023	Osteochondrosis
024	Bone Cyst
025	Bone Cyst
026	Retained Cartilage Core
027	Retained Cartilage Core
028	Normal
029	Normal
030	Bone Lysis
031	Bone Lysis
032	Bone Lysis
033	Radial Hypoplasia
034	Radial Hypoplasia
035	Radial Hypoplasia
036	Radial Hypoplasia
037	Pathologic Fracture
038	Pathologic Fracture
039	Fracture
040	Fracture
041	Normal
042	Normal
043	Bone Lysis
044	Bone Lysis
045	Avulsion
046	Avulsion
047	Normal
048	Normal
049	Luxation
050	Luxation
051	Osteophytes
052	Osteophytes
053	Fracture
054	Fracture
055	Fracture
056	Fracture
057	Frac

446	Normal
447	Normal
448	Normal
449	Normal
450	Normal
451	Soft Tissue Swelling
452	Soft Tissue Swelling
453	Soft Tissue Swelling
454	Soft Tissue Swelling
455	DJD
456	Periosteal Reaction
457	Normal
458	Normal
459	Normal
460	Normal
461	Normal
462	Subluxation
463	Subluxation
464	Implant
465	Normal
466	Normal
467	Normal
468	Normal
469	Normal
470	Deformity
471	Deformity
472	Deformity
473	Deformity
474	DJD
475	DJD
476	Normal
477	Implant
478	Implant
479	Implant
480	DJD
481	DJD
482	DJD
483	Soft Tissue Swelling
484	Soft Tissue Swelling
485	Soft Tissue Swelling
486	Subluxation
487	DJD
488	DJD
489	Subluxation
490	Soft Tissue Swelling
491	Normal
492	Normal
493	Normal
494	Normal
495	Normal
496	Osteosarcoma
497	Osteosarcoma
498	Normal
499	DJD
500	Normal
